# 4. PySpark Mini-project 

## Dataset

Storm events in US during 2019 are studied in the following analysis using the NOAA's National Weather Service Dataset available [here](https://www.ncdc.noaa.gov/stormevents/ftp.jsp). The folder **data** contains three csv files with event details, fatalities and location of every event.

In [1]:
# import SparkContext
from pyspark import SparkContext

In [2]:
sc = SparkSession.builder.appName("pysparkDataframes").getOrCreate()

In [3]:
sqlContext = SQLContext(sc)

In [4]:
dfStorm2019 = sc.read.format('csv')\
                .option('header', 'true')\
                .option('delimiter', ',')\
                .option('inferSchema', 'true')\
                .load('../pyspark/data/StormEvents_details-ftp_v1.0_d2019_c20200317.csv.gz')

In [5]:
dfFatalities2019 = sc.read.format('csv')\
                    .option('header', 'true')\
                    .option('delimiter', ',')\
                    .option('inferSchema', 'true')\
                    .load('../pyspark/data/StormEvents_fatalities-ftp_v1.0_d2019_c20200317.csv.gz')

In [6]:
dfLocations2019 = sc.read.format('csv')\
                    .option('header', 'true')\
                    .option('delimiter', ',')\
                    .option('inferSchema', 'true')\
                    .load('../pyspark/data/StormEvents_locations-ftp_v1.0_d2019_c20200317.csv.gz')

Displaying the schema of every dataset:

In [7]:
dfStorm2019.printSchema()

root
 |-- BEGIN_YEARMONTH: integer (nullable = true)
 |-- BEGIN_DAY: integer (nullable = true)
 |-- BEGIN_TIME: integer (nullable = true)
 |-- END_YEARMONTH: integer (nullable = true)
 |-- END_DAY: integer (nullable = true)
 |-- END_TIME: integer (nullable = true)
 |-- EPISODE_ID: integer (nullable = true)
 |-- EVENT_ID: integer (nullable = true)
 |-- STATE: string (nullable = true)
 |-- STATE_FIPS: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH_NAME: string (nullable = true)
 |-- EVENT_TYPE: string (nullable = true)
 |-- CZ_TYPE: string (nullable = true)
 |-- CZ_FIPS: integer (nullable = true)
 |-- CZ_NAME: string (nullable = true)
 |-- WFO: string (nullable = true)
 |-- BEGIN_DATE_TIME: string (nullable = true)
 |-- CZ_TIMEZONE: string (nullable = true)
 |-- END_DATE_TIME: string (nullable = true)
 |-- INJURIES_DIRECT: integer (nullable = true)
 |-- INJURIES_INDIRECT: integer (nullable = true)
 |-- DEATHS_DIRECT: integer (nullable = true)
 |-- DEATHS_INDIRE

In [8]:
dfFatalities2019.printSchema()

root
 |-- FAT_YEARMONTH: integer (nullable = true)
 |-- FAT_DAY: integer (nullable = true)
 |-- FAT_TIME: integer (nullable = true)
 |-- FATALITY_ID: integer (nullable = true)
 |-- EVENT_ID: integer (nullable = true)
 |-- FATALITY_TYPE: string (nullable = true)
 |-- FATALITY_DATE: string (nullable = true)
 |-- FATALITY_AGE: integer (nullable = true)
 |-- FATALITY_SEX: string (nullable = true)
 |-- FATALITY_LOCATION: string (nullable = true)
 |-- EVENT_YEARMONTH: integer (nullable = true)



In [9]:
dfLocations2019.printSchema()

root
 |-- YEARMONTH: integer (nullable = true)
 |-- EPISODE_ID: integer (nullable = true)
 |-- EVENT_ID: integer (nullable = true)
 |-- LOCATION_INDEX: integer (nullable = true)
 |-- RANGE: double (nullable = true)
 |-- AZIMUTH: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LAT2: integer (nullable = true)
 |-- LON2: integer (nullable = true)



Dataframes have a **registerTempTable attribute** that can be transform into spark sql to generate queries and save the results using **.write.saveAsTable(name_table)**. RDD's don't have this extension because they are not structured tables. 

Obs: **registerTempTable** is deprecated. Instead, use **createOrReplaceTempView**

In [10]:
dfStorm2019.createOrReplaceTempView('stormDetails_table')

### 1. Type of Events

In [11]:
sqlContext.sql('SELECT \
                    EVENT_TYPE, \
                    COUNT(EVENT_TYPE) \
                    FROM stormDetails_table \
                    GROUP BY EVENT_TYPE \
                    ORDER BY COUNT(EVENT_TYPE) DESC').show()

+--------------------+-----------------+
|          EVENT_TYPE|count(EVENT_TYPE)|
+--------------------+-----------------+
|   Thunderstorm Wind|            18617|
|                Hail|             9013|
|               Flood|             4943|
|         Flash Flood|             4068|
|      Winter Weather|             3800|
|           High Wind|             3743|
|        Winter Storm|             3312|
|          Heavy Snow|             2844|
|Marine Thundersto...|             2502|
|             Tornado|             1727|
|         Strong Wind|             1590|
|          Heavy Rain|             1416|
|                Heat|             1291|
|Extreme Cold/Wind...|             1065|
|             Drought|             1007|
|            Blizzard|              852|
|      Excessive Heat|              827|
|        Frost/Freeze|              654|
|           Dense Fog|              652|
|     Cold/Wind Chill|              470|
+--------------------+-----------------+
only showing top

Since 1851 to 2018, hurricanes seasons have hit different states across the country. The top hurricane states on record are Florida, Texas, North Carolina, Luisiana, South Carolina, Alabama, Georgia, Missisipi, New York and Massachussetts. Atlantic Hurricane season runs from June to November, mostly between August and September. From summer to fall, weather conditions become ideal to generate storms later, with cooler air and warm ocean water temperatures. Pacific Hurricane season runs from May to November (the Pacific Coast of the United State is affected by storms originated in Mexico in their roads to come back to the sea, toward Hawaii).

### 2. Number of episodes and events by STATE

One episodes contains an unique or more events happening in different hours and days.

In [12]:
sqlContext.sql('SELECT \
                    STATE, \
                    COUNT(DISTINCT(EPISODE_ID)) AS NUMBER_EPISODES \
                    FROM stormDetails_table \
                    GROUP BY STATE ORDER BY COUNT(DISTINCT(EPISODE_ID))DESC').show(20)

+--------------+---------------+
|         STATE|NUMBER_EPISODES|
+--------------+---------------+
|         TEXAS|            536|
|  SOUTH DAKOTA|            430|
|    CALIFORNIA|            402|
|      COLORADO|            331|
|        KANSAS|            325|
|      ILLINOIS|            299|
|       WYOMING|            294|
|      VIRGINIA|            283|
|GULF OF MEXICO|            280|
|     MINNESOTA|            279|
|      NEBRASKA|            275|
|          IOWA|            263|
|      MISSOURI|            256|
|  PENNSYLVANIA|            256|
|      NEW YORK|            254|
|NORTH CAROLINA|            247|
|       FLORIDA|            243|
|       MONTANA|            232|
|     WISCONSIN|            232|
|          OHIO|            229|
+--------------+---------------+
only showing top 20 rows



The states with more episodes during 2019 were Texas, Soth Dakota, California, Colorado, Kansas

In [13]:
sqlContext.sql('SELECT \
                    STATE, \
                    COUNT(DISTINCT(EVENT_ID)) AS NUMBER_EVENTS \
                    FROM stormDetails_table \
                    GROUP BY STATE ORDER BY COUNT(DISTINCT(EVENT_ID))DESC').show(20)

+--------------+-------------+
|         STATE|NUMBER_EVENTS|
+--------------+-------------+
|         TEXAS|         4338|
|        KANSAS|         2672|
|    CALIFORNIA|         2643|
|  SOUTH DAKOTA|         2543|
|      NEW YORK|         2514|
|      VIRGINIA|         2398|
|  PENNSYLVANIA|         2395|
|          OHIO|         2279|
|          IOWA|         2276|
|      MISSOURI|         2159|
|     MINNESOTA|         2126|
|      NEBRASKA|         2088|
|      ILLINOIS|         2084|
|      OKLAHOMA|         1801|
|      COLORADO|         1776|
|     WISCONSIN|         1573|
|      KENTUCKY|         1522|
|NORTH CAROLINA|         1448|
|       INDIANA|         1447|
|       MONTANA|         1286|
+--------------+-------------+
only showing top 20 rows



In [14]:
sqlContext.sql('SELECT \
                    EVENT_TYPE, \
                    STATE, \
                    COUNT(EVENT_TYPE) AS NUMBER_EVENTS \
                    FROM stormDetails_table \
                    GROUP BY STATE, EVENT_TYPE ORDER BY COUNT(EVENT_TYPE) DESC').show(20)

+--------------------+--------------+-------------+
|          EVENT_TYPE|         STATE|NUMBER_EVENTS|
+--------------------+--------------+-------------+
|                Hail|         TEXAS|         1394|
|   Thunderstorm Wind|  PENNSYLVANIA|         1245|
|   Thunderstorm Wind|      VIRGINIA|         1199|
|   Thunderstorm Wind|         TEXAS|         1047|
|Marine Thundersto...|ATLANTIC NORTH|          997|
|   Thunderstorm Wind|      NEW YORK|          903|
|   Thunderstorm Wind|          OHIO|          891|
|   Thunderstorm Wind|        KANSAS|          848|
|                Hail|        KANSAS|          847|
|   Thunderstorm Wind|NORTH CAROLINA|          791|
|Marine Thundersto...|GULF OF MEXICO|          775|
|               Flood|  SOUTH DAKOTA|          752|
|   Thunderstorm Wind|       GEORGIA|          673|
|   Thunderstorm Wind|      MISSOURI|          648|
|                Hail|      NEBRASKA|          640|
|   Thunderstorm Wind|SOUTH CAROLINA|          619|
|   Thunders

Hails, thunderstorms and marine thonderstoms were the most common events in 2019.

### 3. Duration of events

First, we inspect the date information available in the stormDetails table

In [15]:
sqlContext.sql('SELECT \
                    EPISODE_ID, \
                    EVENT_ID, \
                    STATE, \
                    EVENT_TYPE, \
                    BEGIN_YEARMONTH, \
                    BEGIN_DAY, \
                    BEGIN_TIME,  \
                    END_YEARMONTH, \
                    END_DAY, \
                    END_TIME \
                    FROM stormDetails_table').show(5)

+----------+--------+---------+-----------------+---------------+---------+----------+-------------+-------+--------+
|EPISODE_ID|EVENT_ID|    STATE|       EVENT_TYPE|BEGIN_YEARMONTH|BEGIN_DAY|BEGIN_TIME|END_YEARMONTH|END_DAY|END_TIME|
+----------+--------+---------+-----------------+---------------+---------+----------+-------------+-------+--------+
|    137295|  824116|    TEXAS|      Flash Flood|         201905|        9|      1554|       201905|      9|    1830|
|    140217|  843354|MINNESOTA|Thunderstorm Wind|         201907|       15|      1640|       201907|     15|    1641|
|    142648|  861581|    TEXAS|Thunderstorm Wind|         201910|       20|      2223|       201910|     20|    2223|
|    142648|  861584|    TEXAS|Thunderstorm Wind|         201910|       20|      2312|       201910|     20|    2312|
|    142648|  861582|    TEXAS|Thunderstorm Wind|         201910|       20|      2236|       201910|     20|    2236|
+----------+--------+---------+-----------------+-------

A better datetime visualization results of the transformation of the begin and end dates as strings with format **yyyy-mm-dd hh-mm**

In [16]:
sqlContext.sql("SELECT \
                    EPISODE_ID, \
                    EVENT_ID, \
                    STATE, \
                    EVENT_TYPE, \
                    CONCAT(SUBSTRING(BEGIN_YEARMONTH, 1, 4), '-', \
                    SUBSTRING(BEGIN_YEARMONTH, 5, 2), '-', \
                    BEGIN_DAY, ' ', SUBSTRING(BEGIN_TIME, 1, 2), ':', SUBSTRING(BEGIN_TIME, 3, 2)) AS BEGIN_DATE, \
                    CONCAT(SUBSTRING(END_YEARMONTH, 1, 4), '-', \
                    SUBSTRING(END_YEARMONTH, 5, 2), '-', \
                    END_DAY, ' ', SUBSTRING(END_TIME, 1, 2), ':', SUBSTRING(END_TIME, 3, 2)) AS END_DATE \
                    FROM stormDetails_table").show()   

+----------+--------+--------------+--------------------+----------------+----------------+
|EPISODE_ID|EVENT_ID|         STATE|          EVENT_TYPE|      BEGIN_DATE|        END_DATE|
+----------+--------+--------------+--------------------+----------------+----------------+
|    137295|  824116|         TEXAS|         Flash Flood| 2019-05-9 15:54| 2019-05-9 18:30|
|    140217|  843354|     MINNESOTA|   Thunderstorm Wind|2019-07-15 16:40|2019-07-15 16:41|
|    142648|  861581|         TEXAS|   Thunderstorm Wind|2019-10-20 22:23|2019-10-20 22:23|
|    142648|  861584|         TEXAS|   Thunderstorm Wind|2019-10-20 23:12|2019-10-20 23:12|
|    142648|  861582|         TEXAS|   Thunderstorm Wind|2019-10-20 22:36|2019-10-20 22:36|
|    142648|  856504|         TEXAS|             Tornado|2019-10-20 20:48|2019-10-20 20:54|
|    141212|  848333|       VERMONT|                Hail| 2019-09-4 12:29| 2019-09-4 12:29|
|    141215|  848338|      NEW YORK|   Thunderstorm Wind|2019-09-26 15:54|2019-0

In the table below, `DELTA_YEAR` and `DELTA_MONTH` are the differences between the begin/end years and months respectively. Both are extracted from `BEGIN_YEARMONTH` and `END_YEARMONTH` using the SQL function SUBSTRING. The same attribute is used to retrieve hours and minutes from `BEGIN_TIME` and `END_TIME`. `DELTA_DAYS` is compute directly from `BEGIN_DAY` and `END_DAY`.

100 rows are displayed as follows:

In [17]:
sqlContext.sql('SELECT \
                   EPISODE_ID, \
                   EVENT_ID, STATE, \
                   EVENT_TYPE, \
                   CAST(SUBSTRING(END_YEARMONTH, 1, 4) AS INT) - CAST(SUBSTRING(BEGIN_YEARMONTH, 1, 4) AS INT) AS DELTA_YEAR, \
                   CAST(SUBSTRING(END_YEARMONTH, 5, 2) AS INT) - CAST(SUBSTRING(BEGIN_YEARMONTH, 5, 2) AS INT) AS DELTA_MONTH, \
                   END_DAY - BEGIN_DAY AS DELTA_DAY, \
                   ((CAST(SUBSTRING(END_TIME, 1, 2) AS INT)*60 + CAST(SUBSTRING(END_TIME, 3, 2) AS INT)) - (CAST(SUBSTRING(BEGIN_TIME, 1, 2) AS INT)*60 + CAST(SUBSTRING(BEGIN_TIME, 3, 2) AS INT)))/60.0 AS DELTA_HOURS \
                   FROM stormDetails_table').show()

+----------+--------+--------------+--------------------+----------+-----------+---------+-----------+
|EPISODE_ID|EVENT_ID|         STATE|          EVENT_TYPE|DELTA_YEAR|DELTA_MONTH|DELTA_DAY|DELTA_HOURS|
+----------+--------+--------------+--------------------+----------+-----------+---------+-----------+
|    137295|  824116|         TEXAS|         Flash Flood|         0|          0|        0|   2.600000|
|    140217|  843354|     MINNESOTA|   Thunderstorm Wind|         0|          0|        0|   0.016667|
|    142648|  861581|         TEXAS|   Thunderstorm Wind|         0|          0|        0|   0.000000|
|    142648|  861584|         TEXAS|   Thunderstorm Wind|         0|          0|        0|   0.000000|
|    142648|  861582|         TEXAS|   Thunderstorm Wind|         0|          0|        0|   0.000000|
|    142648|  856504|         TEXAS|             Tornado|         0|          0|        0|   0.100000|
|    141212|  848333|       VERMONT|                Hail|         0|     

Number of events with a `DELTA_MONTH` upper than 0:

In [18]:
sqlContext.sql('SELECT \
                   EPISODE_ID, \
                   EVENT_ID, \
                   STATE, \
                   EVENT_TYPE, \
                   DELTA_DAY, \
                   DELTA_HOURS \
                   FROM (\
                        SELECT \
                            EPISODE_ID, \
                            EVENT_ID, \
                            STATE, \
                            EVENT_TYPE, \
                            CAST(SUBSTRING(END_YEARMONTH, 1, 4) AS INT) - CAST(SUBSTRING(BEGIN_YEARMONTH, 1, 4) AS INT) AS DELTA_YEAR, \
                            CAST(SUBSTRING(END_YEARMONTH, 5, 2) AS INT) - CAST(SUBSTRING(BEGIN_YEARMONTH, 5, 2) AS INT) AS DELTA_MONTH, \
                            END_DAY - BEGIN_DAY AS DELTA_DAY, \
                            ((CAST(SUBSTRING(END_TIME, 1, 2) AS INT)*60 + CAST(SUBSTRING(END_TIME, 3, 2) AS INT)) - (CAST(SUBSTRING(BEGIN_TIME, 1, 2) AS INT)*60 + CAST(SUBSTRING(BEGIN_TIME, 3, 2) AS INT)))/60.0 AS DELTA_HOURS \
                            FROM stormDetails_table \
                        ) \
                   WHERE DELTA_MONTH > 0').show()

+----------+--------+-----+----------+---------+-----------+
|EPISODE_ID|EVENT_ID|STATE|EVENT_TYPE|DELTA_DAY|DELTA_HOURS|
+----------+--------+-----+----------+---------+-----------+
+----------+--------+-----+----------+---------+-----------+



In [19]:
sqlContext.sql('SELECT \
                   B.EPISODE_ID, \
                   B.EVENT_ID, \
                   B.STATE, \
                   B.EVENT_TYPE, \
                   B.BEGIN_YEARMONTH, \
                   B.END_YEARMONTH, \
                   B.BEGIN_DAY, \
                   B.END_DAY, \
                   B.EFFECTIVE_DELTA_DAYS \
                   FROM ( \
                       SELECT \
                           A.EPISODE_ID, \
                           A.EVENT_ID, \
                           A.STATE, \
                           A.EVENT_TYPE, \
                           A.BEGIN_YEARMONTH, \
                           A.END_YEARMONTH, \
                           A.BEGIN_DAY, \
                           A.END_DAY, \
                           CASE WHEN A.DELTA_HOURS IS NOT NULL THEN A.DELTA_DAY + A.DELTA_HOURS/24.0 ELSE A.DELTA_DAY END AS EFFECTIVE_DELTA_DAYS \
                           FROM (\
                                SELECT \
                                   EPISODE_ID, \
                                   EVENT_ID, \
                                   STATE, \
                                   EVENT_TYPE, \
                                   BEGIN_YEARMONTH, \
                                   END_YEARMONTH, \
                                   BEGIN_DAY, \
                                   END_DAY, \
                                   END_DAY - BEGIN_DAY AS DELTA_DAY, \
                                   ABS((CAST(SUBSTRING(END_TIME, 1, 2) AS INT) + CAST(SUBSTRING(END_TIME, 3, 2) AS INT)) - (CAST(SUBSTRING(BEGIN_TIME, 1, 2) AS INT) + CAST(SUBSTRING(BEGIN_TIME, 3, 2) AS INT)))/60.0 AS DELTA_HOURS \
                                   FROM stormDetails_table \
                                   WHERE CAST(END_TIME AS INT) > 0 AND CAST(BEGIN_TIME AS INT) > 0 \
                                ) AS A \
                    ) AS B \
                    ORDER BY B.EFFECTIVE_DELTA_DAYS DESC').show(100)

+----------+--------+--------------+----------+---------------+-------------+---------+-------+--------------------+
|EPISODE_ID|EVENT_ID|         STATE|EVENT_TYPE|BEGIN_YEARMONTH|END_YEARMONTH|BEGIN_DAY|END_DAY|EFFECTIVE_DELTA_DAYS|
+----------+--------+--------------+----------+---------------+-------------+---------+-------+--------------------+
|    140042|  842536|  SOUTH DAKOTA|     Flood|         201907|       201907|        1|     31|       30.0361111250|
|    140042|  842537|  SOUTH DAKOTA|     Flood|         201907|       201907|        1|     31|       30.0361111250|
|    141294|  848629|         IDAHO|  Wildfire|         201908|       201908|        1|     31|       30.0000000000|
|    137101|  822737|      ILLINOIS|     Flood|         201904|       201904|        1|     30|       29.0500000000|
|    138578|  833168|          IOWA|     Flood|         201906|       201906|        1|     30|       29.0500000000|
|    138858|  835315|  SOUTH DAKOTA|     Flood|         201906| 

In [20]:
# INJURIES_DIRECT: integer (nullable = true)
#  |-- INJURIES_INDIRECT: integer (nullable = true)
#  |-- DEATHS_DIRECT: integer (nullable = true)
#  |-- DEATHS_INDIRECT: integer (nullable = true)
#  |-- DAMAGE_PROPERTY: string (nullable = true)
#  |-- DAMAGE_CROPS: string (nullable = true)
#  |-- SOURCE: string (nullable = true)
#  |-- MAGNITUDE: double (nullable = true)
#  |-- MAGNITUDE_TYPE: string (nullable = true)
#  |-- FLOOD_CAUSE: string (nullable = true)
#  |-- CATEGORY: integer (nullable = true)

In [21]:
df_stormPandas = dfStorm2019.toPandas()

In [22]:
sc.stop()